# Import Libraries

In [1]:
import pandas as pd, numpy as np
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import plotly_express as px

sns.set_style("whitegrid") # set some seaborn styles
sns.set_style("ticks")

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 300)
pd.set_option("display.max_colwidth", 500)

# Import Data

In [2]:
df_raw = pd.read_csv("../data_raw/multiple_choice_responses_2019.csv", low_memory=False)

# 1. Review / cleaning 

In [3]:
df_raw["Q3"].value_counts()

India                                                   4786
United States of America                                3085
Other                                                   1054
Brazil                                                   728
Japan                                                    673
Russia                                                   626
China                                                    574
Germany                                                  531
United Kingdom of Great Britain and Northern Ireland     482
Canada                                                   450
Spain                                                    399
Nigeria                                                  395
France                                                   387
Taiwan                                                   301
Turkey                                                   288
Italy                                                    271
Australia               

In [4]:
# get overview, search isnull. 
100 * df_raw.isnull().sum() / len(df_raw)

Time from Start to Finish (seconds)     0.000000
Q1                                      0.000000
Q2                                      0.000000
Q2_OTHER_TEXT                           0.000000
Q3                                      0.000000
Q4                                      1.998174
Q5                                      3.093620
Q5_OTHER_TEXT                           0.000000
Q6                                     28.983670
Q7                                     30.905771
Q8                                     32.914089
Q9_Part_1                              69.104372
Q9_Part_2                              81.909930
Q9_Part_3                              74.733746
Q9_Part_4                              83.015519
Q9_Part_5                              81.204990
Q9_Part_6                              88.031240
Q9_Part_7                              97.301958
Q9_Part_8                              98.732123
Q9_OTHER_TEXT                           0.000000
Q10                 

As most cols are dummy vars we dont drop cols with hight perc isnull

In [5]:
df = df_raw.copy() 

In [6]:
# extract question
questions = df.iloc[0,:]

In [7]:
# drop questions
df = df.drop(df.index[0])

In [8]:
# check and drop duplicates
df = df.drop_duplicates()

In [9]:
# drop columns
## possible to drop later: Q16, Q17, Q20, Q30, Q31, Q32

drop_all = ["Q26", "Time from Start to Finish", "Q11", "Q19"]
#df[df.columns.drop(list(df.filter(regex="Q26", "OTHER", "Time from Start to Finish (seconds)", "Q11", "Q19")))]

for col in drop_all:
    df = df.loc[:,~df.columns.str.contains(col)]
    

In [10]:
# rename cols
rename = {"Q1" : "age",
          "Q2" : "gender",
          "Q3" : "country",
          "Q4" : "degree",
          "Q5" : "position",
          "Q6" : "company_size",
          }
df = df.rename(columns=rename)

# 2. Extract subsets
for further steps we extract the relevant countries

In [11]:
india = df[df["country"] == "India"]

In [12]:
usa = df[df["country"] == "United States of America"]

## 2.1 some plotting

In [13]:
df_tmp = df.copy()
df_tmp["country"] = np.where(df["country"] != np.nan, "global", df["country"])

In [14]:
def perc(df, col):
    
    return round(100 * df[col].value_counts(normalize=True),2).to_frame()

In [15]:
# global_gender_dist = round(100 * df["gender"].value_counts(normalize=True),2).to_frame().rename(columns={"gender" : "global"})
# india_gender_dist = round(100 * india["gender"].value_counts(normalize=True),2).to_frame().rename
# usa_gender_dist = round(100 * usa["gender"].value_counts(normalize=True),2).to_frame()
# global_gender_dist

In [66]:
global_gender_dist = perc(df, "gender").rename(columns={"gender" : "global"})
india_gender_dist = perc(india, "gender").rename(columns={"gender" : "india"})
usa_gender_dist = perc(usa, "gender").rename(columns={"gender" : "usa"})

df_gender = global_gender_dist.merge(right=india_gender_dist, how="inner", left_index=True, right_index=True)
df_gender= df_gender.merge(right=usa_gender_dist, how="inner", left_index=True, right_index=True)

### 2.1.0 Fig Standards

In [17]:
figure_size = (12,6)

### 2.1.1 Gender

In [27]:
# Before we can execute our scripts, we need to connect the JavaScript to our notebook. Since Plotly plots are interactive, they make use of JavaScript behind the scenes. 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

map_data = dict(type='choropleth',
            locations=['IND'],
            locationmode='country names',
            colorscale='India',
            text=['India'],
            z=[1.0,2.0,3.0,4.0],
            colorbar=dict(title="USA States")
           )
map_layout = dict(geo = {'scope':'world'})
map_actual = go.Figure(data=[map_data], layout=map_layout)
iplot(map_actual)

In [62]:
map_data = dict(type='choropleth',
            locations=['INDIA', 'USA'],
            locationmode='country names',
            colorscale='Viridis',
            text=['India'],
            z=[81.84, 77.37],
            colorbar=dict(title="World Gender Distribution"),
)
map_layout = dict(geo = {'scope':'world'})
map_actual = go.Figure(data=[map_data], layout=map_layout)
iplot(map_actual)

In [ ]:
fig , ax = plt.subplots(figsize=figure_size)
sns.barplot(data=df_gender, x=["global", "india"]).set(xlabel="Gender");

### 2.1.2 Age